In [1]:
import selenium 
import csv
import re
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.chrome.options import Options

import time

In [2]:
sample_df = pd.read_csv("cas_get_inxight_add.csv")
# sample_df = sample_df.sample(200)

cas_df = sample_df[sample_df["cas"] != "No CAS info"]
missing_df = sample_df[sample_df["cas"] == "No CAS info"] 
sample_df

,molecule,parsed_molecule,cas
0,XYLANASE,XYLANASE,"9068-42-2, 9025-57-4, 7554-16-7"
1,WOOL_FAT,WOOL FAT,8006-54-0
2,RHAMNUS_PURSHIANA,RHAMNUS PURSHIANA,8015-89-2
3,PROPANTHELINE_HYDROXIDE,PROPANTHELINE HYDROXIDE,298-50-0
4,PIPER_METHYSTICUM,PIPER METHYSTICUM,9000-38-8
5,OXALIPLATIN,OXALIPLATIN,61825-94-3
6,HYPROMELLOSE,HYPROMELLOSE,9004-65-3
7,HYPROLOSE,HYPROLOSE,9004-64-2
8,EPTACOG_ALFA_(ACTIVATED),EPTACOG ALFA (ACTIVATED),102786-61-8
9,COLESTYRAMINE,COLESTYRAMINE,11041-12-6


In [3]:
cas_df = cas_df[['parsed_molecule', 'cas']]
missing_df = missing_df[['parsed_molecule', 'cas']]

In [4]:
cas_df

,parsed_molecule,cas
0,XYLANASE,"9068-42-2, 9025-57-4, 7554-16-7"
1,WOOL FAT,8006-54-0
2,RHAMNUS PURSHIANA,8015-89-2
3,PROPANTHELINE HYDROXIDE,298-50-0
4,PIPER METHYSTICUM,9000-38-8
5,OXALIPLATIN,61825-94-3
6,HYPROMELLOSE,9004-65-3
7,HYPROLOSE,9004-64-2
8,EPTACOG ALFA (ACTIVATED),102786-61-8
9,COLESTYRAMINE,11041-12-6


In [5]:
missing_df

,parsed_molecule,cas


In [6]:
def get_first_cas(cas_string):
    # Replace all semicolons with commas
    modified_string = re.sub(r';', ',', cas_string)
    
    if modified_string == "No CAS info":
        return None
    
    return modified_string.split(',')[0].strip()

# Apply this function to the entire CAS column
cas_df['cas'] = cas_df['cas'].apply(get_first_cas)

molecules = list(zip(cas_df['parsed_molecule'], cas_df['cas']))
molecules

# missing_df['CAS'] = missing_df['CAS'].apply(get_first_cas)
# missing_arr = missing_df['CAS'].tolist()

[('XYLANASE', '9068-42-2'),
 ('WOOL FAT', '8006-54-0'),
 ('RHAMNUS PURSHIANA', '8015-89-2'),
 ('PROPANTHELINE HYDROXIDE', '298-50-0'),
 ('PIPER METHYSTICUM', '9000-38-8'),
 ('OXALIPLATIN', '61825-94-3'),
 ('HYPROMELLOSE', '9004-65-3'),
 ('HYPROLOSE', '9004-64-2'),
 ('EPTACOG ALFA (ACTIVATED)', '102786-61-8'),
 ('COLESTYRAMINE', '11041-12-6'),
 ('CARBONYL IRON', '7439-89-6'),
 ('CALASPARGASE PEGOL', '941577-06-6'),
 ('CACAO BUTTER', '8002-31-1'),
 ('BROMELAINS', '9001-00-7'),
 ('ACACIA SENEGAL', '9000-01-5')]

In [7]:
import urllib

def setup_webdriver():
    '''Initializes a headless selenium webdriver'''
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

def cas_url_inxight(molecule):
    cas = molecule[1]
    cas_formatted = cas.replace('-', '%20')
    return f'https://drugs.ncats.io/substances?q=(root_codes_CAS:\"{cas_formatted}\")'

def name_url_inxight(molecule):
    '''Generates an Inxight URL for a given CAS'''
    name = molecule[0]
    cas = molecule[1]
    cas_formatted = cas.replace('-', '%20')
    name_encoded = urllib.parse.quote(f'^{name}')
    return f'https://drugs.ncats.io/substances?q=(root_codes_CAS:\"{cas_formatted}\")%20AND%20(root_names_name:\"{name_encoded}\")'

def name_exact_inxight(molecule):
    '''Generates an Inxight URL for a given CAS'''
    name = molecule[0]
    cas = molecule[1]
    cas_formatted = cas.replace('-', '%20')
    name_encoded = urllib.parse.quote(f'^{name}')
    return f'https://drugs.ncats.io/substances?q=(root_codes_CAS:\"{cas_formatted}\")%20AND%20(root_names_name:\"{name_encoded}$\")'

In [8]:
def get_inxight_url(molecule, driver):
    '''
    Parameters
    ----------
    A molecule's CAS and an initialised webdriver.

    Returns
    -------
    The top Inxight search result for a given CAS number
    '''
    
    url = "N/A"  # Default in case of failure
    query = cas_url_inxight(molecule)
    query_type = "ONLY CAS"
    driver.get(query)
    
    try:
        WebDriverWait(driver, 3).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'span#record-count:nth-child(2)')))
        elements = driver.find_elements(By.CSS_SELECTOR, 'span#record-count:nth-child(2)')
        if elements:
            count = int(elements[0].text)
            print(count)
        else:
            print("Element not found.")
            
    except TimeoutException:
        print("Element not found within specified time.")
        count = 0

    if count > 1:
        query = name_exact_inxight(molecule)
        query_type = "EXACT NAME"
        driver.get(query)
    else:
        pass
        
    try:
        WebDriverWait(driver, 3).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'a[id="card-title"]')))
        elements = driver.find_elements(By.CSS_SELECTOR, 'a[id="card-title"]')
        if elements:
            element = elements[0]
            url = element.get_attribute('href')

    except (NoSuchElementException, TimeoutException):
        query = name_url_inxight(molecule)
        query_type = "APPROXIMATE NAME"
        driver.get(query)
        
        try:
            WebDriverWait(driver, 3).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'a[id="card-title"]')))
            elements = driver.find_elements(By.CSS_SELECTOR, 'a[id="card-title"]')
            if elements:
                element = elements[0]
                url = element.get_attribute('href')

        except (NoSuchElementException, TimeoutException):
            query = cas_url_inxight(molecule)
            query_type = "ONLY CAS"
            driver.get(query)

            try:
                WebDriverWait(driver, 3).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'a[id="card-title"]')))
                elements = driver.find_elements(By.CSS_SELECTOR, 'a[id="card-title"]')
                if elements:
                    element = elements[0]
                    url = element.get_attribute('href')
            
            except (NoSuchElementException, TimeoutException):
                pass
            
    return molecule, url, count, query, query_type

def get_best_urls(mol_arr):
    '''
    Parameters
    ----------
    An array of CAS values

    Returns
    -------
    An array of associated top matches
    '''
    
    best_matches = []
    driver = setup_webdriver()  
    try:
        for molecule in mol_arr:
            molecule, url, count, query, query_type = get_inxight_url(molecule, driver)
            best_matches.append((molecule, url, count, query, query_type))
    finally:
        driver.quit()  

    return best_matches

In [9]:
import requests

def get_additional_data(id):
    r = requests.get(f'https://drugs.ncats.io/api/v1/substances({id})/@additional')
    if 200 == r.status_code:
        return r.json()
    return None

In [10]:
def extract_conditions_and_phases(data):
    conditions_list = []
    highest_approval_list = []

    if data is None:
        return conditions_list, highest_approval_list

    for entry in data:
        if entry['name'] == 'Conditions' and 'value' in entry:
            condition_info = entry['value']
            
            # Extract the condition name
            label = condition_info.get('label')
            if label:
                conditions_list.append(label)
            
            # Extract the highest phase of approval
            highest_phase = condition_info.get('highestPhase')
            if highest_phase:
                highest_approval_list.append(highest_phase)

    return conditions_list, highest_approval_list

In [11]:
# def extract_event_details(data):
#     event_details = {}
    
#     for item in data:
#         if 'value' in item and isinstance(item['value'], dict):  # Ensure 'value' is a dictionary
#             details = item['value']
#             if 'status' in details and 'sourceID' in details:
#                 if item['name'] == 'Highest Development Event' or item['name'] == 'Earliest Approved Event':
#                     # Gather additional details
#                     source_id = details.get('sourceID', 'No Source ID')
#                     source_url = details.get('sourceURL', 'No Source URL')
                    
#                     detail_info = {
#                         'Status and Year': f"{details['status']} {details.get('year', '')}",
#                         'Source ID': source_id,
#                         'Source URL': source_url
#                     }
                    
#                     # Use the 'name' of the event as the key in the dictionary
#                     event_details[item['name']] = detail_info
    
#     return event_details

def extract_event_details(data):
    event_details = {}
    if data is None:  # Check if data is None before iterating
        return event_details

    for item in data:
        if 'value' in item and isinstance(item['value'], dict):
            details = item['value']
            if 'status' in details and 'sourceID' in details:
                if item['name'] in ['Highest Development Event', 'Earliest Approved Event']:
                    detail_info = {
                        'Status and Year': f"{details['status']} {details.get('year', '')}",
                        'Source ID': details.get('sourceID', 'No Source ID'),
                        'Source URL': details.get('sourceURL', 'No Source URL')
                    }
                    event_details[item['name']] = detail_info
    return event_details

In [12]:
links = get_best_urls(molecules)
links

Element not found within specified time.
2
7
1
Element not found within specified time.
1
25
18
1
1
4
1
1
3
1


[(('XYLANASE', '9068-42-2'),
  'N/A',
  0,
  'https://drugs.ncats.io/substances?q=(root_codes_CAS:"9068%2042%202")',
  'ONLY CAS'),
 (('WOOL FAT', '8006-54-0'),
  'https://drugs.ncats.io/drug/7EV65EAW6H',
  2,
  'https://drugs.ncats.io/substances?q=(root_codes_CAS:"8006%2054%200")%20AND%20(root_names_name:"%5EWOOL%20FAT$")',
  'EXACT NAME'),
 (('RHAMNUS PURSHIANA', '8015-89-2'),
  'https://drugs.ncats.io/drug/4VBP01X99F',
  7,
  'https://drugs.ncats.io/substances?q=(root_codes_CAS:"8015%2089%202")%20AND%20(root_names_name:"%5ERHAMNUS%20PURSHIANA$")',
  'EXACT NAME'),
 (('PROPANTHELINE HYDROXIDE', '298-50-0'),
  'https://drugs.ncats.io/drug/1306V2B0Q8',
  1,
  'https://drugs.ncats.io/substances?q=(root_codes_CAS:"298%2050%200")',
  'ONLY CAS'),
 (('PIPER METHYSTICUM', '9000-38-8'),
  'N/A',
  0,
  'https://drugs.ncats.io/substances?q=(root_codes_CAS:"9000%2038%208")',
  'ONLY CAS'),
 (('OXALIPLATIN', '61825-94-3'),
  'https://drugs.ncats.io/drug/04ZR38536J',
  1,
  'https://drugs.ncats.

In [13]:
# new_links_df = pd.DataFrame(links, columns=['molecule', 'query_url', 'results', 'best_match_url', ''])
# new_links_df.head(2)
# links = new_links_df['best_match_url'].to_list()
# links
# identifiers = [url.split('/')[-1] for _, url in links]
# identifiers

data = []
for link in links:
    molecule_name, cas_number = link[0]
    drug_url, value, substance_url, query_type = link[1:]
    data.append([molecule_name, cas_number, drug_url, value, substance_url, query_type])
    
new_links_df = pd.DataFrame(data, columns=['molecule', 'cas', 'best_match_url', 'results', 'query_url', 'query_type'])
new_links_df.head(2)

,molecule,cas,best_match_url,results,query_url,query_type
0,XYLANASE,9068-42-2,N/A,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS
1,WOOL FAT,8006-54-0,https://drugs.ncats.io/drug/7EV65EAW6H,2,https://drugs.ncats.io/substances?q=(root_code...,EXACT NAME


In [14]:
identifier_arr = new_links_df['best_match_url'].to_list()
identifier_arr
identifiers = [url.split('/')[-1] for url in identifier_arr]

In [15]:
identifiers = [identifier if identifier != 'A' else 'MISSING' for identifier in identifiers]

In [16]:
len(identifiers)

15

In [17]:
def extract_events(identifiers):
    data = []
    events = []

    for identifier in identifiers:
        data = get_additional_data(identifier)
        print(extract_event_details(data))
        events.append(extract_event_details(data))

    return events
        
events = extract_events(identifiers)

# def extract_events(identifiers):
#     events = []
#     for identifier in identifiers:
#         data = get_additional_data(identifier)
#         if data is not None:  # Check if data is None
#             event_details = extract_event_details(data)
#             events.append(event_details)
#         else:
#             print(f"No data available for identifier {identifier}")  # Or handle it differently
#     return events

{}
{'Highest Development Event': {'Status and Year': 'US Approved OTC ', 'Source ID': '21 CFR 349.14(a)(2) ophthalmic:emollient lanolin  (in combination)', 'Source URL': 'https://www.gpo.gov/fdsys/pkg/CFR-2018-title21-vol5/xml/CFR-2018-title21-vol5-sec349-14.xml'}, 'Earliest Approved Event': {'Status and Year': 'US Previously Marketed 1921', 'Source ID': 'Hydrous Wool Fat U.S.P.', 'Source URL': 'https://www.gutenberg.org/files/41778/41778-h/41778-h.htm'}}
{'Highest Development Event': {'Status and Year': 'US Previously Marketed ', 'Source ID': '21 CFR 310.545(a)(12)(iv)(C) laxative:stimulant laxative cascara sagrada bark', 'Source URL': 'https://www.gpo.gov/fdsys/pkg/CFR-2018-title21-vol5/xml/CFR-2018-title21-vol5-sec310-545.xml'}}
{'Highest Development Event': {'Status and Year': 'US Previously Marketed 1982', 'Source ID': 'PROPANTHELINE BROMIDE by HEATHER', 'Source URL': 'https://www.accessdata.fda.gov/scripts/cder/daf/index.cfm?event=overview.process&ApplNo=085780'}, 'Earliest Appro

In [18]:
def extract_conditions(identifiers):
    conditions = []

    for identifier in identifiers:
        data = get_additional_data(identifier)
        conditions.append(extract_conditions_and_phases(data))

    return conditions
        
conditions_arr = extract_conditions(identifiers)

In [19]:
events

[{},
 {'Highest Development Event': {'Status and Year': 'US Approved OTC ',
   'Source ID': '21 CFR 349.14(a)(2) ophthalmic:emollient lanolin  (in combination)',
   'Source URL': 'https://www.gpo.gov/fdsys/pkg/CFR-2018-title21-vol5/xml/CFR-2018-title21-vol5-sec349-14.xml'},
  'Earliest Approved Event': {'Status and Year': 'US Previously Marketed 1921',
   'Source ID': 'Hydrous Wool Fat U.S.P.',
   'Source URL': 'https://www.gutenberg.org/files/41778/41778-h/41778-h.htm'}},
 {'Highest Development Event': {'Status and Year': 'US Previously Marketed ',
   'Source ID': '21 CFR 310.545(a)(12)(iv)(C) laxative:stimulant laxative cascara sagrada bark',
   'Source URL': 'https://www.gpo.gov/fdsys/pkg/CFR-2018-title21-vol5/xml/CFR-2018-title21-vol5-sec310-545.xml'}},
 {'Highest Development Event': {'Status and Year': 'US Previously Marketed 1982',
   'Source ID': 'PROPANTHELINE BROMIDE by HEATHER',
   'Source URL': 'https://www.accessdata.fda.gov/scripts/cder/daf/index.cfm?event=overview.process

In [20]:
conditions_arr

[([], []),
 ([], []),
 ([], []),
 (['Overactive bladder', 'Urinary calculus', 'Peptic ulcer'],
  ['Approved', 'Phase IV', 'Approved']),
 ([], []),
 (['Carcinoma of the colon or rectum', 'Colon cancer'],
  ['Approved', 'Approved']),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], [])]

In [21]:
data = []
for event in events:
    record = {}
    for key, value in event.items():
        for sub_key, sub_value in value.items():
            record[f"{key} {sub_key}"] = sub_value
    data.append(record)

df = pd.DataFrame(data)

In [22]:
df

,Highest Development Event Status and Year,Highest Development Event Source ID,Highest Development Event Source URL,Earliest Approved Event Status and Year,Earliest Approved Event Source ID,Earliest Approved Event Source URL
0,NaN,NaN,NaN,NaN,NaN,NaN
1,US Approved OTC,21 CFR 349.14(a)(2) ophthalmic:emollient lanol...,https://www.gpo.gov/fdsys/pkg/CFR-2018-title21...,US Previously Marketed 1921,Hydrous Wool Fat U.S.P.,https://www.gutenberg.org/files/41778/41778-h/...
2,US Previously Marketed,21 CFR 310.545(a)(12)(iv)(C) laxative:stimulan...,https://www.gpo.gov/fdsys/pkg/CFR-2018-title21...,NaN,NaN,NaN
3,US Previously Marketed 1982,PROPANTHELINE BROMIDE by HEATHER,https://www.accessdata.fda.gov/scripts/cder/da...,US Previously Marketed 1953,PRO-BANTHINE by SHIRE,https://www.accessdata.fda.gov/scripts/cder/da...
4,NaN,NaN,NaN,NaN,NaN,NaN
5,US Approved Rx 2017,ANDA205529,https://www.accessdata.fda.gov/scripts/cder/da...,US Previously Marketed 2002,ELOXATIN by SANOFI AVENTIS US,https://www.accessdata.fda.gov/scripts/cder/da...
6,Possibly Marketed Outside US,Canada:HYPROMELLOSE,https://tripod.nih.gov/npc/#Download,Possibly Marketed Outside US 1960,ANDA040424,https://dailymed.nlm.nih.gov/dailymed/drugInfo...
7,US Approved Rx 1981,NDA018771,https://www.accessdata.fda.gov/scripts/cder/da...,Possibly Marketed Outside US 1975,NDA017579,https://dailymed.nlm.nih.gov/dailymed/drugInfo...
8,Possibly Marketed Outside US,Niastase RT by Novo Nordisk [Canada],https://www.drugbank.ca/drugs/DB00036,Possibly Marketed Outside US 1999,BLA103665,https://dailymed.nlm.nih.gov/dailymed/drugInfo...
9,US Approved Rx 2020,ANDA209599,https://www.accessdata.fda.gov/scripts/cder/da...,US Previously Marketed 1964,Cuemid by Merck Sharp & Dohme,DeHaen 1940-1975 NMEs


In [23]:
rows = [{'conditions': conditions, 'phases': phases} for conditions, phases in conditions_arr]

# Create DataFrame
conditions_df = pd.DataFrame(rows)
conditions_df['conditions'] = conditions_df['conditions'].apply(lambda x: '; '.join(x))
conditions_df['phases'] = conditions_df['phases'].apply(lambda x: '; '.join(x))
conditions_df

,conditions,phases
0,,
1,,
2,,
3,Overactive bladder; Urinary calculus; Peptic u...,Approved; Phase IV; Approved
4,,
5,Carcinoma of the colon or rectum; Colon cancer,Approved; Approved
6,,
7,,
8,,
9,,


In [24]:
merged = pd.concat([df, conditions_df], axis=1)
merged

,Highest Development Event Status and Year,Highest Development Event Source ID,Highest Development Event Source URL,Earliest Approved Event Status and Year,Earliest Approved Event Source ID,Earliest Approved Event Source URL,conditions,phases
0,NaN,NaN,NaN,NaN,NaN,NaN,,
1,US Approved OTC,21 CFR 349.14(a)(2) ophthalmic:emollient lanol...,https://www.gpo.gov/fdsys/pkg/CFR-2018-title21...,US Previously Marketed 1921,Hydrous Wool Fat U.S.P.,https://www.gutenberg.org/files/41778/41778-h/...,,
2,US Previously Marketed,21 CFR 310.545(a)(12)(iv)(C) laxative:stimulan...,https://www.gpo.gov/fdsys/pkg/CFR-2018-title21...,NaN,NaN,NaN,,
3,US Previously Marketed 1982,PROPANTHELINE BROMIDE by HEATHER,https://www.accessdata.fda.gov/scripts/cder/da...,US Previously Marketed 1953,PRO-BANTHINE by SHIRE,https://www.accessdata.fda.gov/scripts/cder/da...,Overactive bladder; Urinary calculus; Peptic u...,Approved; Phase IV; Approved
4,NaN,NaN,NaN,NaN,NaN,NaN,,
5,US Approved Rx 2017,ANDA205529,https://www.accessdata.fda.gov/scripts/cder/da...,US Previously Marketed 2002,ELOXATIN by SANOFI AVENTIS US,https://www.accessdata.fda.gov/scripts/cder/da...,Carcinoma of the colon or rectum; Colon cancer,Approved; Approved
6,Possibly Marketed Outside US,Canada:HYPROMELLOSE,https://tripod.nih.gov/npc/#Download,Possibly Marketed Outside US 1960,ANDA040424,https://dailymed.nlm.nih.gov/dailymed/drugInfo...,,
7,US Approved Rx 1981,NDA018771,https://www.accessdata.fda.gov/scripts/cder/da...,Possibly Marketed Outside US 1975,NDA017579,https://dailymed.nlm.nih.gov/dailymed/drugInfo...,,
8,Possibly Marketed Outside US,Niastase RT by Novo Nordisk [Canada],https://www.drugbank.ca/drugs/DB00036,Possibly Marketed Outside US 1999,BLA103665,https://dailymed.nlm.nih.gov/dailymed/drugInfo...,,
9,US Approved Rx 2020,ANDA209599,https://www.accessdata.fda.gov/scripts/cder/da...,US Previously Marketed 1964,Cuemid by Merck Sharp & Dohme,DeHaen 1940-1975 NMEs,,


In [25]:
new_links_df

,molecule,cas,best_match_url,results,query_url,query_type
0,XYLANASE,9068-42-2,N/A,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS
1,WOOL FAT,8006-54-0,https://drugs.ncats.io/drug/7EV65EAW6H,2,https://drugs.ncats.io/substances?q=(root_code...,EXACT NAME
2,RHAMNUS PURSHIANA,8015-89-2,https://drugs.ncats.io/drug/4VBP01X99F,7,https://drugs.ncats.io/substances?q=(root_code...,EXACT NAME
3,PROPANTHELINE HYDROXIDE,298-50-0,https://drugs.ncats.io/drug/1306V2B0Q8,1,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS
4,PIPER METHYSTICUM,9000-38-8,N/A,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS
5,OXALIPLATIN,61825-94-3,https://drugs.ncats.io/drug/04ZR38536J,1,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS
6,HYPROMELLOSE,9004-65-3,https://drugs.ncats.io/drug/3NXW29V3WO,25,https://drugs.ncats.io/substances?q=(root_code...,EXACT NAME
7,HYPROLOSE,9004-64-2,https://drugs.ncats.io/drug/9XZ8H6N6OH,18,https://drugs.ncats.io/substances?q=(root_code...,EXACT NAME
8,EPTACOG ALFA (ACTIVATED),102786-61-8,https://drugs.ncats.io/drug/AC71R787OV,1,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS
9,COLESTYRAMINE,11041-12-6,https://drugs.ncats.io/drug/4B33BGI082,1,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS


In [26]:
result = pd.concat([new_links_df, merged], axis=1)
result

,molecule,cas,best_match_url,results,query_url,query_type,Highest Development Event Status and Year,Highest Development Event Source ID,Highest Development Event Source URL,Earliest Approved Event Status and Year,Earliest Approved Event Source ID,Earliest Approved Event Source URL,conditions,phases
0,XYLANASE,9068-42-2,N/A,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,NaN,NaN,NaN,NaN,NaN,NaN,,
1,WOOL FAT,8006-54-0,https://drugs.ncats.io/drug/7EV65EAW6H,2,https://drugs.ncats.io/substances?q=(root_code...,EXACT NAME,US Approved OTC,21 CFR 349.14(a)(2) ophthalmic:emollient lanol...,https://www.gpo.gov/fdsys/pkg/CFR-2018-title21...,US Previously Marketed 1921,Hydrous Wool Fat U.S.P.,https://www.gutenberg.org/files/41778/41778-h/...,,
2,RHAMNUS PURSHIANA,8015-89-2,https://drugs.ncats.io/drug/4VBP01X99F,7,https://drugs.ncats.io/substances?q=(root_code...,EXACT NAME,US Previously Marketed,21 CFR 310.545(a)(12)(iv)(C) laxative:stimulan...,https://www.gpo.gov/fdsys/pkg/CFR-2018-title21...,NaN,NaN,NaN,,
3,PROPANTHELINE HYDROXIDE,298-50-0,https://drugs.ncats.io/drug/1306V2B0Q8,1,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,US Previously Marketed 1982,PROPANTHELINE BROMIDE by HEATHER,https://www.accessdata.fda.gov/scripts/cder/da...,US Previously Marketed 1953,PRO-BANTHINE by SHIRE,https://www.accessdata.fda.gov/scripts/cder/da...,Overactive bladder; Urinary calculus; Peptic u...,Approved; Phase IV; Approved
4,PIPER METHYSTICUM,9000-38-8,N/A,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,NaN,NaN,NaN,NaN,NaN,NaN,,
5,OXALIPLATIN,61825-94-3,https://drugs.ncats.io/drug/04ZR38536J,1,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,US Approved Rx 2017,ANDA205529,https://www.accessdata.fda.gov/scripts/cder/da...,US Previously Marketed 2002,ELOXATIN by SANOFI AVENTIS US,https://www.accessdata.fda.gov/scripts/cder/da...,Carcinoma of the colon or rectum; Colon cancer,Approved; Approved
6,HYPROMELLOSE,9004-65-3,https://drugs.ncats.io/drug/3NXW29V3WO,25,https://drugs.ncats.io/substances?q=(root_code...,EXACT NAME,Possibly Marketed Outside US,Canada:HYPROMELLOSE,https://tripod.nih.gov/npc/#Download,Possibly Marketed Outside US 1960,ANDA040424,https://dailymed.nlm.nih.gov/dailymed/drugInfo...,,
7,HYPROLOSE,9004-64-2,https://drugs.ncats.io/drug/9XZ8H6N6OH,18,https://drugs.ncats.io/substances?q=(root_code...,EXACT NAME,US Approved Rx 1981,NDA018771,https://www.accessdata.fda.gov/scripts/cder/da...,Possibly Marketed Outside US 1975,NDA017579,https://dailymed.nlm.nih.gov/dailymed/drugInfo...,,
8,EPTACOG ALFA (ACTIVATED),102786-61-8,https://drugs.ncats.io/drug/AC71R787OV,1,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,Possibly Marketed Outside US,Niastase RT by Novo Nordisk [Canada],https://www.drugbank.ca/drugs/DB00036,Possibly Marketed Outside US 1999,BLA103665,https://dailymed.nlm.nih.gov/dailymed/drugInfo...,,
9,COLESTYRAMINE,11041-12-6,https://drugs.ncats.io/drug/4B33BGI082,1,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,US Approved Rx 2020,ANDA209599,https://www.accessdata.fda.gov/scripts/cder/da...,US Previously Marketed 1964,Cuemid by Merck Sharp & Dohme,DeHaen 1940-1975 NMEs,,


In [27]:
sample_output = result.to_csv("cas_inxight_added.csv", index = False)